In [13]:
# Import libraries
import pandas as pd
import numpy  as np
import os
from scipy import stats
import docx
from sentence_transformers import SentenceTransformer, models

In [14]:
meta = pd.read_csv("align_overview_sum - align_overview_sum.csv")
data = meta.loc[(meta['Arousal_binary'] != -1) & (meta['Valence_binary'] != -1) & (meta['check date'] != 0)].reset_index(drop=True)
data['ID'] = data['audio'].str[:5]
fspeak = ["P_001","P_002","P_003","P_004","P_005","P_006","P_007","P_008","P_009","P_010","P_011","Z_012","P_013"]
mspeak = ["Z_001","Z_002","Z_003","Z_004","Z_005","Z_006","Z_007","Z_008","Z_009","Z_010","Z_011","P_012","Z_013"]
data = data.loc[(data["female speak"] == "yes") & (data["male speak"] == "yes")].reset_index(drop=True)

In [15]:
# Create transcript paths and assign gender

meta_data = data.copy()
fspeak = ["P_001","P_002","P_003","P_004","P_005","P_006","P_007","P_008","P_009","P_010","P_011","Z_012","P_013"]
mspeak = ["Z_001","Z_002","Z_003","Z_004","Z_005","Z_006","Z_007","Z_008","Z_009","Z_010","Z_011","P_012","Z_013"]
meta_data.loc[meta_data['ID'].isin(fspeak), 'Gender'] = "f"
meta_data.loc[meta_data['ID'].isin(mspeak), 'Gender'] = "m"

In [16]:
meta_data

,audio,contain speech,female speak,male speak,female transcript,male transcript,annotate,useful annotates,female chunk number,aligned female chunk number,...,Arousal_binary,Valence_binary,check annotation,check female transcript,check male transcript,check date,romantic interaction,romantic interaction & both speak,ID,Gender
0,P_001\Day_2\Hour_16\07-16-2019 16_44_00,yes,yes,yes,1,1,1,5,2,2,...,1,1,1,1,1,1,1,1,P_001,f
1,P_001\Day_2\Hour_18\07-16-2019 18_24_27,yes,yes,yes,1,1,1,29,18,12,...,1,1,1,1,1,1,1,1,P_001,f
2,P_001\Day_2\Hour_9\07-16-2019 09_00_26,yes,yes,yes,1,1,0,-1,7,-1,...,1,1,0,1,1,1,1,1,P_001,f
3,P_001\Day_3\Hour_16\07-17-2019 16_06_50,yes,yes,yes,1,1,1,13,20,9,...,1,1,1,1,1,1,1,1,P_001,f
4,P_001\Day_3\Hour_17\07-17-2019 17_02_08,yes,yes,yes,1,1,1,46,18,15,...,1,1,1,1,1,1,1,1,P_001,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Z_012\Day_1\Hour_9\08-09-2021 09_21_24,yes,yes,yes,1,1,1,46,18,18,...,1,1,1,1,1,1,0,-1,Z_012,f
376,Z_013\Day_4\Hour_21\10-21-2021 21_31_33,yes,yes,yes,1,1,1,1,12,1,...,0,1,1,1,1,1,1,1,Z_013,m
377,Z_013\Day_3\Hour_21\10-20-2021 21_44_00,yes,yes,yes,1,1,1,4,5,2,...,0,1,1,1,1,1,1,1,Z_013,m
378,Z_013\Day_2\Hour_21\10-19-2021 21_12_32,yes,yes,yes,1,1,1,28,19,18,...,1,1,1,1,1,1,1,1,Z_013,m


In [17]:
splits =splits = meta_data['audio'].str.split("\\").str.join("_").str.split("_")
name = splits.str[0:2].str.join("")
day = splits.str[2:5].str.join("_")
hour = splits.str[5].str.zfill(2)
time = splits.str[2:6].str.join("_")
date = splits.str[6:].str.join("_")

meta_data['transcript_path'] = "transcript_" + name + "_" + time + "-" + date + "_" + meta_data['Gender']
# meta_data['transcript_path'] = "transcript_" + name + "_" + day + "_" + hour + "-" + date + "_" + meta_data['Gender']

# Getting full path 
meta_data['transcript_full_path'] = "Z:\\DyMand Data\\Transcripts\\" + meta_data['transcript_path'].str[11:15] + "\\" + meta_data['transcript_path'] + ".docx"

In [18]:
transcript_path = meta_data['transcript_full_path'].tolist()
gender_type = meta_data['Gender'].tolist()

In [19]:
def convertToText(file):
    # Load file
    load_doc = docx.Document(file)
    
    text = []
    # Iterate over paragraphs
    for para in load_doc.paragraphs:
        
        # Preprocessing
        text_filtered = para.text.replace("//","").replace("XX","").replace("XY","").replace("\n\n","")
        
        if text_filtered and text_filtered.strip():
            text.append(text_filtered)
    
    
    trans = ",".join(text)
    
    return trans

In [22]:
# Create csv of transcripts
transcripts = []
skips = []
for path, gender in zip(transcript_path, gender_type):
    
    try:
        trans_file = convertToText(path)
    except:
        print("Skipping file:", path)
        skips.append(path)
        continue
        
#     file_process = path.split("\\")[-1][11:29]
#     filename = file_process[0] + "_" + file_process[1:]


    file_process = path.split("\\")[-1].split("_")[1:6]
    hour = file_process[4].split("-")[0]
    name = file_process[0][0] + "_" + file_process[0][1:] +"_" + "_".join(file_process[1:4])
    filename = name + "_" + hour
    file = [filename, gender, trans_file]
    transcripts.append(file)
    
transcripts_df = pd.DataFrame(transcripts, columns = ['Name', 'Gender','Transcript'])



Skipping file: Z:\DyMand Data\Transcripts\Z007\transcript_Z007_Day_5_Hour_10-02-14-2020 10_44_24_m.docx


In [23]:
transcripts_df.to_csv("transcript/transcript.csv", index=False)

# Remember to run DyMand Linguistic features before evaluating model

In [12]:
transcripts_df

,Name,Gender,Transcript
0,P_001_Day_2_Hour_16,f,Du musst dann da noch vom Raf.. Raffi… Schak....
1,P_001_Day_2_Hour_18,f,Danke. Ist jetzt bei dir noch immer nichts gek...
2,P_001_Day_3_Hour_16,f,"Man könnte ja, aber du. Nicht, nein. Die komme..."
3,P_001_Day_3_Hour_17,f,"Das ist höher oben. Mhm ist ja hier, geknecht..."
4,P_001_Day_3_Hour_19,f,Und für alle diese Jahre sei die Versicherung ...
...,...,...,...
373,Z_012_Day_1_Hour_9,f,"Ja. Das ist schon doof, mit dieser Frisur hier..."
374,Z_013_Day_4_Hour_21,m,Du kannst . Ich muss schauen wegen meinter M...
375,Z_013_Day_3_Hour_21,m,Ich habe deine Socken alle gewendet um einen d...
376,Z_013_Day_2_Hour_21,m,schon gemerkt. Im Örlikon getroffen. Weil S...


## Filtering for correct speakers

In [34]:
# Applying filtering to remove incorrect speakers

fspeak = ["P001","P002","P003","P004","P005","P006","P007","P008","P009","P010","P011","Z012","P013"]
mspeak = ["Z001","Z002","Z003","Z004","Z005","Z006","Z007","Z008","Z009","Z010","Z011","P012","Z013"]
transcripts_df['ID'] = transcripts_df['Name'].str.split("_").str[:2].str.join("")
transcriptions_df_filtered = transcripts_df.loc[(transcripts_df['ID'].isin(fspeak)) & (transcripts_df["Gender"] == "f") | (transcripts_df['ID'].isin(mspeak)) & (transcripts_df["Gender"] == "m")]

In [39]:
transcriptions_df_filtered.reset_index(drop=True,inplace=True)
transcriptions_df_filtered
transcriptions_df_filtered.to_csv("transcript/transcript.csv", index=False)

## Finding skipped transcripts

In [37]:
# Find the skipped transcripts
trans_skipped = pd.DataFrame(skips, columns = ['transcript'])
trans_skipped['gender'] = trans_skipped['transcript'].str[-6:-5]
trans_skipped['ID'] = trans_skipped['transcript'].str.split("\\").str[3]
fspeak = ["P001","P002","P003","P004","P005","P006","P007","P008","P009","P010","P011","Z012","P013"]
mspeak = ["Z001","Z002","Z003","Z004","Z005","Z006","Z007","Z008","Z009","Z010","Z011","P012","Z013"]
transcriptions_skipped_gender_filtered = trans_skipped.loc[(trans_skipped['ID'].isin(fspeak)) & (trans_skipped["gender"] == "f") | (trans_skipped['ID'].isin(mspeak)) & (trans_skipped["gender"] == "m")]
transcriptions_skipped_gender_filtered.reset_index(drop=True, inplace=True)
transcriptions_skipped_gender_filtered['transcript']
# transcriptions_skipped_gender_filtered.to_csv("transcriptions_skipped_priority.csv", index=False)

0    Z:\DyMand Data\Transcripts\P001\transcript_P00...
1    Z:\DyMand Data\Transcripts\P004\transcript_P00...
2    Z:\DyMand Data\Transcripts\P004\transcript_P00...
3    Z:\DyMand Data\Transcripts\P005\transcript_P00...
4    Z:\DyMand Data\Transcripts\P009\transcript_P00...
5    Z:\DyMand Data\Transcripts\P009\transcript_P00...
6    Z:\DyMand Data\Transcripts\Z004\transcript_Z00...
7    Z:\DyMand Data\Transcripts\Z004\transcript_Z00...
8    Z:\DyMand Data\Transcripts\Z006\transcript_Z00...
9    Z:\DyMand Data\Transcripts\Z007\transcript_Z00...
Name: transcript, dtype: object